In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
import pandas as pd
import os

# Starts a Spark session
spark = (
    SparkSession.builder
        .appName("Taxi vs Rideshare Profitability")
        .config("spark.sql.repl.eagerEval.enabled", False)   
        .config("spark.sql.parquet.cacheMetadata", "true")
        .config("spark.sql.session.timeZone", "Etc/UTC")
        .config("spark.sql.shuffle.partitions", "320")
        .config("spark.sql.adaptive.enabled", "true") 
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true")       
        .config("spark.driver.memory", "6g")                
        .config("spark.executor.memory", "6g")
        .getOrCreate()
)

# Define months
months = ["2024-01","2024-02","2024-03","2024-04","2024-05","2024-06"]

# Load in data files 
yellow_files = [f"data/yellow/yellow_tripdata_{m}.parquet" for m in months]
fhvhv_files  = [f"data/fhvhv/fhvhv_tripdata_{m}.parquet"   for m in months]

df_yellow = (
    spark.read.parquet(*yellow_files)
         .withColumn("service_type", lit("yellow"))
)
df_fhvhv = (
    spark.read.parquet(*fhvhv_files)
         .withColumn("service_type", lit("hv_fhv"))
)

# Merge
df = df_yellow.unionByName(df_fhvhv, allowMissingColumns=True)

# External tables 
electricity = spark.read.csv("data/external/electricity.csv", header=True, inferSchema=True)
fuel = spark.read.csv("data/external/fuel.csv", header=True, inferSchema=True)



Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/25 18:55:42 WARN Utils: Your hostname, LAPTOP-E04ANIN1, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/25 18:55:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/25 18:55:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:

# Preprocess data
from pyspark.sql import functions as F
from pyspark.sql.functions import col, to_timestamp, coalesce, unix_timestamp, when, lit, date_format, hour, dayofweek, broadcast
from pyspark.sql.types import IntegerType, DoubleType
from pyspark import StorageLevel
from pyspark.sql import Window
import geopandas as gpd

# Print size of raw data
print(f"Raw data count: {df.count():,}")

# Reduce memory usage
_needed = [
    "service_type",
    "tpep_pickup_datetime","tpep_dropoff_datetime",
    "pickup_datetime","dropoff_datetime",
    "trip_distance","trip_miles","trip_time",
    "PULocationID","DOLocationID",
    "passenger_count","payment_type",
    "fare_amount","extra","tip_amount",
    "driver_pay","tips"
]
df = df.select([c for c in _needed if c in df.columns])

# Standardise timestamps
df = (
    df.withColumn("pickup_ts",  to_timestamp(coalesce(col("tpep_pickup_datetime"),  col("pickup_datetime"))))
      .withColumn("dropoff_ts", to_timestamp(coalesce(col("tpep_dropoff_datetime"), col("dropoff_datetime"))))
)

# Remove rows with null pickup or dropoff timestamps
df = df.filter(col("pickup_ts").isNotNull() & col("dropoff_ts").isNotNull())

# Standardise location IDs
df = df.withColumn("month", date_format(col("pickup_ts"), "yyyy-MM"))

# Standardise distance and keep positive distances only and not null
df = (
    df.withColumn("distance_mi", coalesce(col("trip_distance"), col("trip_miles")).cast(DoubleType()))
      .filter(col("distance_mi").isNotNull() & (col("distance_mi") > 0))
)

# Standardise trip time and not null
df = df.withColumn(
    "trip_time_s",
    when(col("trip_time").isNotNull(), col("trip_time").cast("double"))
    .otherwise((unix_timestamp(col("dropoff_ts")) - unix_timestamp(col("pickup_ts"))).cast("double"))
)
df = df.filter(col("trip_time_s").isNotNull() & (col("trip_time_s") > 0))

# Deduplicate rows 
dedupe_key = [c for c in ["service_type","pickup_ts","dropoff_ts","PULocationID","DOLocationID","distance_mi","trip_time_s"] if c in df.columns]

# If dedupe_key is empty, we won't deduplicate
if dedupe_key:
    w = Window.partitionBy(["month"] + dedupe_key).orderBy(F.lit(1))
    df = df.withColumn("__rn", F.row_number().over(w)).filter(col("__rn") == 1).drop("__rn")

# Fixed Parameters
CREDIT_CARD_FEE = 0.025 
MAINTENANCE_COST_PER_MILE = 0.15
MAINTENANCE_COST_PER_MILE_HV = 0.15

# Payment type exists
if "payment_type" not in df.columns:
    df = df.withColumn("payment_type", lit(None).cast(IntegerType()))

# Time features 
df = (df
    .withColumn("pickup_hour", hour(col("pickup_ts")))
    .withColumn("pickup_dow", dayofweek(col("pickup_ts")))  
    .withColumn("is_weekend", (col("pickup_dow").isin([1,7])).cast("boolean"))
)

# Add revenue 
rev_yellow = coalesce(col("fare_amount"), lit(0.0)) + coalesce(col("extra"), lit(0.0)) + coalesce(col("tip_amount"), lit(0.0))
rev_hv     = coalesce(col("driver_pay"), lit(0.0)) + coalesce(col("tips"), lit(0.0))
df = df.withColumn(
    "revenue",
    when(col("service_type") == "yellow", rev_yellow).otherwise(rev_hv).cast(DoubleType())
)

# Add costs
maint_rate = when(col("service_type") == "yellow",
                  lit(MAINTENANCE_COST_PER_MILE)
              ).otherwise(
                  lit(MAINTENANCE_COST_PER_MILE_HV)
              )
df = df.withColumn("expense_maintenance", (col("distance_mi") * maint_rate).cast(DoubleType()))

# Credit card fee 
df = df.withColumn(
     "expense_cc_processing",
    when((col("service_type") == "yellow") & (col("payment_type") == 1),
         (lit(CREDIT_CARD_FEE) * col("revenue")).cast(DoubleType()))
    .otherwise(lit(0.0))
)

# Expenses pre-fuel 
df = df.withColumn(
    "expenses_nonfuel",
    (col("expense_maintenance") + col("expense_cc_processing")).cast(DoubleType())
)

# Keep distance
df = df.filter(col("distance_mi") >= 0.1)

# Make sure within month range
df = df.filter( (col("month") >= "2024-01") & (col("month") <= "2024-06") )

# Keep duration >= 60 seconds
df = df.filter(col("trip_time_s") >= 60)

# Keep positive passenger count 
if "passenger_count" in df.columns:
    df = df.filter(
        when(col("service_type") == "yellow", col("passenger_count") > 0)
        .otherwise(True)
    )

TAXI_ZONES_PATH = "data/taxi_zones/taxi_zones.shp"
zones_gdf = gpd.read_file(TAXI_ZONES_PATH)[["LocationID"]]

valid_ids = (
    zones_gdf["LocationID"]
      .dropna()
      .astype("int64")
      .unique()
      .tolist()
)
# Valid TLC zone IDs 
for c in ["PULocationID", "DOLocationID"]:
    if c in df.columns:
        df = df.filter(col(c).isin(valid_ids))

# Non-negative money fields 
money_ok = (
    (coalesce(col("fare_amount"), lit(0.0))  >= 0) &
    (coalesce(col("extra"),       lit(0.0))  >= 0) &
    (coalesce(col("tip_amount"),  lit(0.0))  >= 0) &
    (coalesce(col("driver_pay"),  lit(0.0))  >= 0) &
    (coalesce(col("tips"),        lit(0.0))  >= 0)
)
df = df.filter(money_ok)

# Minimum initial fare for Yellow 
df = df.filter(
    when(col("service_type") == "yellow", coalesce(col("fare_amount"), lit(0.0)) >= 1.50)
    .otherwise(True)
)


# Pre-fuel profitability
df = (df
    .withColumn("active_hours", (col("trip_time_s") / 3600.0).cast(DoubleType()))
    .withColumn("net_before_fuel", (col("revenue") - col("expenses_nonfuel")).cast(DoubleType()))
    .withColumn("net_per_hr_before_fuel", (col("net_before_fuel") / col("active_hours")).cast(DoubleType()))
    .withColumn("mph", (col("distance_mi") / (col("trip_time_s")/3600.0)).cast(DoubleType()))
)

# Single-pass outlier trim using 99.9% quantile
stacked = (
    df.select("service_type", F.lit("distance_mi").alias("metric"), col("distance_mi").alias("value"))
      .unionByName(df.select("service_type", F.lit("trip_time_s").alias("metric"), col("trip_time_s").alias("value")))
      .unionByName(df.select("service_type", F.lit("revenue").alias("metric"),     col("revenue").alias("value")))
)
bounds = (
    stacked.groupBy("service_type", "metric")
           .agg(F.expr("percentile_approx(value, 0.999, 10000)").alias("p999"))
)
df = (
    df.alias("t")
      .join(bounds.alias("b1").filter(col("b1.metric") == "distance_mi")
                 .select(col("service_type").alias("s1"), col("p999").alias("p_d")),
            on=[col("t.service_type") == col("s1")], how="left")
      .join(bounds.alias("b2").filter(col("b2.metric") == "trip_time_s")
                 .select(col("service_type").alias("s2"), col("p999").alias("p_t")),
            on=[col("t.service_type") == col("s2")], how="left")
      .join(bounds.alias("b3").filter(col("b3.metric") == "revenue")
                 .select(col("service_type").alias("s3"), col("p999").alias("p_r")),
            on=[col("t.service_type") == col("s3")], how="left")
      .filter( (col("distance_mi") <= F.coalesce(col("p_d"), lit(float("inf")))) &
               (col("trip_time_s") <= F.coalesce(col("p_t"), lit(float("inf")))) &
               (col("revenue")     <= F.coalesce(col("p_r"), lit(float("inf")))) )
      .drop("s1","s2","s3","p_d","p_t","p_r")
)

df = df.repartition(64, "service_type", "month")

# Cap impossible speeds
df = df.filter((col("mph") >= 0) & (col("mph") <= 120.0))


# Fuel and energy costs
fuel = fuel.select("month", "price_per_gallon").dropna()
electricity = electricity.select("month", "price_usd_per_kwh").dropna()

# Join fuel and electricity prices
df = (df.join(broadcast(fuel), on="month", how="left")
        .join(broadcast(electricity), on="month", how="left")
)
# Remove if fuel or electricity price is missing
df = df.filter(col("price_per_gallon").isNotNull() & col("price_usd_per_kwh").isNotNull())

# Energy assumptions according to EPA and AFDC 
MPG_FHV  = 27.0  
MPG_TAXI = 16.0  

KWH_YELLOW = 0.30
KWH_FHV    = 0.30

YELLOW_EV_PERCENT = 0.00  # Assuming no EVs in Yellow Taxi fleet 
FHV_EV_PERCENT    = 0.10  # Example share for HVFHV

# Per-service parameters as columns 
df = (df
    .withColumn(
        "mpg",
        when(col("service_type") == "yellow", lit(MPG_TAXI))
        .otherwise(lit(MPG_FHV)).cast(DoubleType())
    )
    .withColumn(
        "kwh_per_mile",
        when(col("service_type") == "yellow", lit(KWH_YELLOW))
        .otherwise(lit(KWH_FHV)).cast(DoubleType())
    )
    .withColumn(
        "ev_share",
        when(col("service_type") == "yellow", lit(YELLOW_EV_PERCENT))
        .otherwise(lit(FHV_EV_PERCENT)).cast(DoubleType())
    )
)

# Cost per mile (blend gas vs EV by ev_share)
gas_cpm = (col("price_per_gallon") / col("mpg")).cast(DoubleType())
ev_cpm  = (col("price_usd_per_kwh") * col("kwh_per_mile")).cast(DoubleType())

df = (df
    .withColumn(
        "energy_cost_per_mile",
        ((lit(1.0) - coalesce(col("ev_share"), lit(0.0))) * coalesce(gas_cpm, lit(0.0))) +
        (coalesce(col("ev_share"), lit(0.0)) * coalesce(ev_cpm, lit(0.0)))
    )
    .withColumn("expense_fuel", (col("distance_mi") * col("energy_cost_per_mile")).cast(DoubleType()))
    .withColumn("net_after_fuel", (col("revenue") - col("expenses_nonfuel") - col("expense_fuel")).cast(DoubleType()))
    .withColumn("net_per_hr_after_fuel", (col("net_after_fuel") / col("active_hours")).cast(DoubleType()))
)

# Keep only relevant columns
cols_keep = [
    # Metadata
    "service_type", "month", "pickup_ts", "dropoff_ts",
    "pickup_hour", "pickup_dow", "is_weekend",
    # Location IDs
    "PULocationID", "DOLocationID",
    # engineered trip metrics
    "distance_mi", "trip_time_s", "mph", "active_hours",
    # Feature engineering
    "revenue", "expenses_nonfuel", "expense_fuel",
    "net_before_fuel", "net_after_fuel",
    "net_per_hr_before_fuel", "net_per_hr_after_fuel",
    # Parameters
    "price_per_gallon", "price_usd_per_kwh",
    "energy_cost_per_mile", "ev_share", "mpg", "kwh_per_mile",
]

# Filter columns to keep only those that exist in the DataFrame
cols_keep = [c for c in cols_keep if c in df.columns]

# Print size after cleaning
df = df.select(cols_keep)
print(f"Cleaned data count: {df.count():,}")

Raw data count: 141,196,761


Cleaned data count: 132,803,826


In [3]:
# Analyze results 
from pyspark.sql.functions import col, sum as ssum, count, round as sround, when, lit
from pyspark.sql import functions as F
from pyspark import StorageLevel

# Testing
FAST_MODE = False  
if FAST_MODE:
    df_base = df.stat.sampleBy("service_type", {"yellow": 0.02, "hv_fhv": 0.02}, seed=7)
else:
    df_base = df

df_base = df.repartition(320, "service_type", "month")

# Aggregate functions
sum_net   = ssum("net_after_fuel").alias("sum_net")
sum_hours = ssum("active_hours").alias("sum_hours")

def safe_net_per_hr(df_agg):
    return df_agg.withColumn(
        "net_per_hr_TW",
        sround(F.when(col("sum_hours") > 0, col("sum_net")/col("sum_hours")), 2)
    )

# Overall net per hour
overall = (
    df_base.groupBy("service_type")
           .agg(sum_net, sum_hours, count("*").alias("trips"))
)
overall = safe_net_per_hr(overall)


# Hour of day
hod = (
    df_base.groupBy("service_type", "pickup_hour")
           .agg(sum_net, sum_hours, count("*").alias("trips"))
)
hod = safe_net_per_hr(hod).orderBy("service_type", "pickup_hour")


# Earnings per trip
comp = (
    df_base.groupBy("service_type")
           .agg(
               ssum("revenue").alias("sum_rev"),
               ssum("expenses_nonfuel").alias("sum_nonfuel"),
               ssum("expense_fuel").alias("sum_fuel"),
               count("*").alias("n_trips")
           )
           .withColumn("rev_per_trip",     sround(col("sum_rev")/col("n_trips"), 2))
           .withColumn("nonfuel_per_trip", sround(col("sum_nonfuel")/col("n_trips"), 2))
           .withColumn("fuel_per_trip",    sround(col("sum_fuel")/col("n_trips"), 2))
           .select("service_type","n_trips","rev_per_trip","nonfuel_per_trip","fuel_per_trip")
)


# Pickup density map
zone_trips = (
    df_base.groupBy("service_type","PULocationID")
           .agg(count("*").alias("trips"))
           .orderBy("service_type", col("trips").desc())
)

zone_pivot = (
    df_base.groupBy("PULocationID","service_type").count()
           .groupBy("PULocationID")
           .pivot("service_type", ["yellow","hv_fhv"])
           .agg(F.first("count"))
           .fillna(0)
           .withColumn("diff_pickups", (col("yellow") - col("hv_fhv")).cast("int"))
)



In [4]:
# Plotting results

import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import LineString
import os
from matplotlib.ticker import FuncFormatter

os.makedirs("plots/img", exist_ok=True)

# Overall Table
overall_pd = overall.toPandas()
print("\nOVERALL (time-weighted net/hr):")
print(overall_pd)

# Hour-of-day Plot
hod_pd = hod.toPandas()
hod_p = (
    hod_pd.pivot(index="pickup_hour", columns="service_type", values="net_per_hr_TW")
          .reindex(range(24))  
          .sort_index()
)

fig, ax = plt.subplots(figsize=(8,4)) 
hod_p.plot(ax=ax, marker="o", linewidth=1.8, title="Time-weighted Net Profit by Hour of Day")

ax.set_xlabel("Pickup time")
ax.set_ylabel("Net $ per active hour")
ax.yaxis.set_major_formatter(FuncFormatter(lambda v, _: f"${v:,.0f}"))

major_xticks = list(range(0, 24, 2))
ax.set_xticks(major_xticks)
ax.set_xticklabels([f"{h:02d}:00" for h in major_xticks], rotation=0)

ax.set_xticks(range(24), minor=True)                  
ax.grid(True, axis="y", alpha=0.3)                    
ax.grid(True, which="minor", axis="x", alpha=0.12)    

ax.set_ylim(bottom=0)
ax.axhline(0, linewidth=1)

plt.tight_layout()
plt.savefig("plots/img/hour_of_day.png", dpi=150)
plt.close()

# Composition
comp_pd = comp.toPandas().set_index("service_type")[["rev_per_trip","nonfuel_per_trip","fuel_per_trip"]]

ax = comp_pd.plot(kind="bar", stacked=True, title="Per-trip Revenue vs Costs")
ax.set_xlabel("Service type")              
ax.set_ylabel("USD per trip")              
ax.yaxis.set_major_formatter(FuncFormatter(lambda v, _: f"${v:,.0f}")) 

ax.set_ylim(bottom=0)      
ax.axhline(0, linewidth=1) 
ax.grid(True, axis="y", alpha=0.3)
plt.xticks(rotation=0)

plt.tight_layout()
plt.savefig("plots/img/composition.png", dpi=150)
plt.close()



OVERALL (time-weighted net/hr):
  service_type       sum_net     sum_hours      trips  net_per_hr_TW
0       hv_fhv  2.083564e+09  3.641863e+07  115610585          57.21
1       yellow  3.807237e+08  4.614187e+06   17193243          82.51


In [5]:
# GEOSPATIAL
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.colors import TwoSlopeNorm

# Load NYC Taxi Zones shapefile
TAXI_ZONES_PATH = "data/taxi_zones/taxi_zones.shp"
zones = gpd.read_file(TAXI_ZONES_PATH)[["LocationID","geometry","zone","borough"]]
zones["LocationID"] = pd.to_numeric(zones["LocationID"], errors="coerce").astype("Int64")

# Difference between yellow and hv_fhv pickups
zone_diff_pd = zone_pivot.toPandas().rename(
    columns={"PULocationID": "LocationID", "diff_pickups": "pickup_diff"}
)
zone_diff_pd["LocationID"] = pd.to_numeric(zone_diff_pd["LocationID"], errors="coerce").astype("Int64")
zone_diff_pd["pickup_diff"] = pd.to_numeric(zone_diff_pd["pickup_diff"], errors="coerce")

z_diff = zones.merge(zone_diff_pd[["LocationID","pickup_diff"]], on="LocationID", how="left")

vals = z_diff["pickup_diff"].astype(float).to_numpy()
vmax = float(pd.Series(vals).abs().max()) if len(vals) else 1.0
norm = TwoSlopeNorm(vmin=-vmax, vcenter=0, vmax=vmax)

fig, ax = plt.subplots(figsize=(8, 6))
z_diff.plot(
    column="pickup_diff",
    cmap="RdBu_r",
    legend=True,
    norm=norm,
    linewidth=0.2,
    edgecolor="black",
    missing_kwds={"color": "lightgrey"},
    ax=ax
)

ax.set_title("Pickup Difference (Yellow − HVFHV)")
ax.set_axis_off()  

cax = fig.axes[-1]
cax.set_ylabel("Pickups (difference)")
cax.yaxis.set_major_formatter(mtick.StrMethodFormatter("{x:,.0f}"))

plt.tight_layout()
plt.savefig("plots/img/map_pickups_diff.png", dpi=150)
plt.close()


In [6]:
# Machine learning model 1

# Regression model to predict net profit per hour
from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import avg
from pyspark.ml.evaluation import RegressionEvaluator

# Define features and target
features = [
    "PULocationID","pickup_hour","is_weekend","month","price_per_gallon","price_usd_per_kwh"
]
target = "net_per_hr_after_fuel"

# Prepare data for ML
df_ml = df.select("service_type", *features, target).dropna()
df_ml = df_ml.withColumn("is_weekend", col("is_weekend").cast("int"))

# Train-test split on june data
train = df_ml.filter(col("month") < "2024-06")
test  = df_ml.filter(col("month") == "2024-06")

# Hot encoding
idx_service = StringIndexer(inputCol="service_type", outputCol="service_type_idx", handleInvalid="keep")
idx_month = StringIndexer(inputCol="month", outputCol="month_idx", handleInvalid="keep")
idx_location = StringIndexer(inputCol="PULocationID", outputCol="PULocationID_idx", handleInvalid="keep")
ohe = OneHotEncoder(
    inputCols=["service_type_idx", "month_idx", "PULocationID_idx"],
    outputCols=["service_type_ohe", "month_ohe", "PULocationID_ohe"],
    handleInvalid="keep", dropLast=True
)

# Assembler for features
assembler = VectorAssembler(
    inputCols=["pickup_hour", "is_weekend", "price_per_gallon", "price_usd_per_kwh",
               "service_type_ohe", "month_ohe", "PULocationID_ohe"],
    outputCol="features"
)
# Linear regression model
linear = LinearRegression(
    featuresCol="features",
    labelCol=target,
    maxIter=200,
    regParam=0.1,
    elasticNetParam=0.8,
)

# Pipeline
pipeline = Pipeline(stages=[idx_service, idx_month, idx_location, ohe, assembler, linear])

# Fit the model
model = pipeline.fit(train)
predictions = model.transform(test)

# Evaluate the model
e_rmse = RegressionEvaluator(
    labelCol=target,
    predictionCol="prediction",
    metricName="rmse"
)
e_mae = RegressionEvaluator(
    labelCol=target,
    predictionCol="prediction",
    metricName="mae"
)
e_r2 = RegressionEvaluator(
    labelCol=target,
    predictionCol="prediction",
    metricName="r2"
)
print(f"RMSE={e_rmse.evaluate(predictions):.2f}  MAE={e_mae.evaluate(predictions):.2f}  R²={e_r2.evaluate(predictions):.3f}")

25/08/25 21:00:18 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/08/25 21:27:21 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


RMSE=28.25  MAE=14.74  R²=0.141


In [7]:
# ML Plot 1
from pyspark.sql import functions as F

avg_pred = (predictions.groupBy("PULocationID","service_type")
            .agg(F.avg("prediction").alias("avg_pred")))

wide = (avg_pred.groupBy("PULocationID")
        .pivot("service_type")
        .agg(F.first("avg_pred")))

# Adjust service_type names to match yours
svc_cols = [c for c in wide.columns if c != "PULocationID"]
yellow_col = next((c for c in svc_cols if "yellow" in c.lower()), None)
ride_col   = next((c for c in svc_cols if "hv" in c.lower() or "fhv" in c.lower()), None)

pred_diff = (wide.withColumn("profit_diff", F.col(ride_col) - F.col(yellow_col))
             .select("PULocationID","profit_diff")
             .toPandas()
             .rename(columns={"PULocationID":"LocationID"}))

pred_diff["LocationID"] = pd.to_numeric(pred_diff["LocationID"], errors="coerce").astype("Int64")

# Merge predictions with zone shapefile
z_diff = zones.merge(pred_diff, on="LocationID", how="left")

# --- Plot ---
vals = z_diff["profit_diff"].astype(float).to_numpy()
vmax = float(pd.Series(vals).abs().max()) if len(vals) else 1.0
norm = TwoSlopeNorm(vmin=-vmax, vcenter=0, vmax=vmax)

fig, ax = plt.subplots(figsize=(8, 6))
z_diff.plot(
    column="profit_diff",
    cmap="RdBu_r",
    legend=True,
    norm=norm,
    linewidth=0.2,
    edgecolor="black",
    missing_kwds={"color": "lightgrey"},
    ax=ax
)

ax.set_title("Predicted Net Profit Difference ($/hr, HVFHV − Yellow)")
ax.set_axis_off()

cax = fig.axes[-1]
cax.set_ylabel("Δ $/hr (HVFHV − Yellow)")
cax.yaxis.set_major_formatter(mtick.StrMethodFormatter("{x:,.2f}"))

plt.tight_layout()
plt.savefig("plots/img/map_profit_diff.png", dpi=150)
plt.close()

In [8]:
# Machine learning model 2

from pyspark.sql.functions import avg as Favg
from pyspark.ml.stat import Summarizer
import numpy as np
from pyspark.ml.feature import VectorAssembler

# Build a feature vector for the second model
assembler_2 = VectorAssembler(
    inputCols=["month_ohe", "PULocationID_ohe", "pickup_hour", "is_weekend", "price_per_gallon", "price_usd_per_kwh"],
    outputCol="features_2"
)

# Reuse the fitted pipeline
train_pp = model.transform(train)
test_pp = model.transform(test)

train2 = assembler_2.transform(train_pp).select("service_type", "features_2", target)
test2 = assembler_2.transform(test_pp).select("service_type", "features_2", target)

# OLS regression model
ols = LinearRegression(
    featuresCol="features_2",
    labelCol=target,
    maxIter=200,
    regParam=0.0,
    elasticNetParam=0.8,
)

m_y = ols.fit(train2.filter(col("service_type")=="yellow"))
m_h = ols.fit(train2.filter(col("service_type")=="hv_fhv"))

βy, b0y = np.array(m_y.coefficients.toArray()), float(m_y.intercept)
βh, b0h = np.array(m_h.coefficients.toArray()), float(m_h.intercept)

# Feature importance
mean_y = np.array(test2.filter(col("service_type")=="yellow").select(Summarizer.mean(col("features_2")).alias("m")).first()[0])
mean_h = np.array(test2.filter(col("service_type")=="hv_fhv").select(Summarizer.mean(col("features_2")).alias("m")).first()[0])
                                                      
# Observed gap
ybarY = float(test.filter(col("service_type")=="yellow").select(Favg(target)).first()[0])
ybarH = float(test.filter(col("service_type")=="hv_fhv").select(Favg(target)).first()[0])
gap   = ybarY - ybarH

# Two-way decomposition
mix = float(np.dot((mean_y - mean_h), βh))
unexp = float(np.dot(mean_y, (βy - βh)) + (b0y - b0h))

print(f"[OAXACA] Observed gap (Yellow − HVFHV): {gap:.2f}")
print(f"[OAXACA] Explained by mix:              {mix:.2f}")
print(f"[OAXACA] Unexplained (service effect):  {unexp:.2f}")
if abs(gap) > 1e-12:
    print(f"[OAXACA] % mix: {100*mix/gap:5.1f}%   |   % unexplained: {100*unexp/gap:5.1f}%")


25/08/26 09:26:37 WARN Instrumentation: [8247b24a] regParam is zero, which might cause numerical instability and overfitting.
25/08/26 09:28:20 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK
25/08/26 09:28:20 WARN Instrumentation: [8247b24a] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.
25/08/26 10:13:09 WARN Instrumentation: [a2d62443] regParam is zero, which might cause numerical instability and overfitting.
25/08/26 10:15:40 WARN Instrumentation: [a2d62443] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


[OAXACA] Observed gap (Yellow − HVFHV): 31.84
[OAXACA] Explained by mix:              2.74
[OAXACA] Unexplained (service effect):  30.21
[OAXACA] % mix:   8.6%   |   % unexplained:  94.9%


In [9]:
import os, numpy as np, matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

os.makedirs("plots/img", exist_ok=True)

fig, ax = plt.subplots(figsize=(7,4))

# bars
ax.bar(0, gap, color="#bdbdbd", label="Observed gap")
ax.bar(1, mix,   color="#1f77b4", label="Explained by mix")
ax.bar(1, unexp, bottom=mix, color="#ff7f0e", label="Unexplained (service effect)")

# axis & title
ax.axhline(0, color="black", lw=0.8, alpha=0.7)
ax.set_xticks([0,1])
ax.set_xticklabels(["Observed", "Decomposition"])
ax.set_ylabel("USD per active hour")
ax.yaxis.set_major_formatter(FuncFormatter(lambda v, _: f"${v:,.0f}"))
ax.set_title("Yellow − HVFHV Net Earnings Gap (Oaxaca–Blinder)")

# annotations
ax.text(0, gap*0.5, f"${gap:,.2f}", ha="center", va="center",
        fontsize=12, color="white", fontweight="bold")

if abs(gap) > 1e-12:
    ax.text(1, mix*0.5, f"{(mix/gap)*100:,.1f}%\n(${mix:,.2f})",
            ha="center", va="center", color="white", fontsize=11)
    ax.text(1, mix + unexp*0.5, f"{(unexp/gap)*100:,.1f}%\n(${unexp:,.2f})",
            ha="center", va="center", color="white", fontsize=11)

ax.legend(frameon=False, loc="upper left")
plt.tight_layout()
plt.savefig("plots/img/model2_oaxaca_bar_v2.png", dpi=200)
plt.close()
